In [ ]:
from eyened_orm import ImageInstance,  Database
from sqlalchemy import select, func
from random import sample
from eyened_orm.importer.thumbnails import get_thumbnail

In [ ]:
database = Database('../../dev/.env')
session = database.create_session()

In [ ]:
images = session.scalars(select(ImageInstance).order_by(func.random()).limit(2000)).all()

In [ ]:
images_binary = [im for im in images if im.path.suffix == ".binary"]
images_dicom = [im for im in images if im.path.suffix == ".dcm"]
images_png = [im for im in images if im.path.suffix == ".png"]

In [ ]:
def plot(ims_sample):
    import numpy as np

    import matplotlib.pyplot as plt
    
    fig, axs = plt.subplots(2, 4, figsize=(15, 8))
    axs = axs.flatten()

    for i, (im) in enumerate(ims_sample):
        print(im.ImageInstanceID, im.path)
        img = get_thumbnail(im)
        if img is not None:
            if img.ndim == 3:
                axs[i].imshow(img)
            else:
                axs[i].imshow(img, cmap="gray")
            
        axs[i].set_title(f"ID: {im.ImageInstanceID}")
        axs[i].axis('off')

    plt.tight_layout()
    plt.show()

In [ ]:
ims_sample = sample(images_png, 8)
plot(ims_sample)

In [ ]:
ims_sample = sample(images_dicom, 8)
plot(ims_sample)

In [ ]:
ims_sample = sample(images_binary, 8)
plot(ims_sample)